In [1]:
import pandas as pd
import  os
from datetime import datetime
import sqlite3


cleaning_log = {
    'initial_row_count': None,
    'rows_removed_negative_duration': 0,
    'rows_removed_over_24h': 0,
    'final_row_count': None,
    'pct_removed': None,
    'output_files_saved': []
}

In [2]:
raw_data_path = '../data/raw/'
all_files = [os.path.join(raw_data_path, f) for f in os.listdir(raw_data_path) if f.endswith('.csv')]
print("Reading and combining all CSV files...")
df_list = [pd.read_csv(file) for file in all_files]
combined_df  = pd.concat(df_list , ignore_index = True)

cleaning_log['initial_row_count'] = combined_df.shape[0]
print(f'Successfully combined {len(all_files)} CSV files. into DataFrame with {cleaning_log["initial_row_count"]} rows.')

Reading and combining all CSV files...
Successfully combined 12 CSV files. into DataFrame with 5860568 rows.


In [3]:
checkpoint_path = '../data/clean/01_combined_raw.parquet'
combined_df.to_parquet(checkpoint_path, index=False, engine='pyarrow')
cleaning_log['output_files_saved'].append(checkpoint_path)
print(f'Raw checkpoint saved to {checkpoint_path}.')

Raw checkpoint saved to ../data/clean/01_combined_raw.parquet.


In [4]:
combined_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C1D650626C8C899A,electric_bike,2024-01-12 15:30:27,2024-01-12 15:37:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.903267,-87.634737,41.889177,-87.638506,member
1,EECD38BDB25BFCB0,electric_bike,2024-01-08 15:45:46,2024-01-08 15:52:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902937,-87.634440,41.889177,-87.638506,member
2,F4A9CE78061F17F7,electric_bike,2024-01-27 12:27:19,2024-01-27 12:35:19,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902951,-87.634470,41.889177,-87.638506,member
3,0A0D9E15EE50B171,classic_bike,2024-01-29 16:26:17,2024-01-29 16:56:06,Wells St & Randolph St,TA1305000030,Larrabee St & Webster Ave,13193,41.884295,-87.633963,41.921822,-87.644140,member
4,33FFC9805E3EFF9A,classic_bike,2024-01-31 05:43:23,2024-01-31 06:09:35,Lincoln Ave & Waveland Ave,13253,Kingsbury St & Kinzie St,KA1503000043,41.948797,-87.675278,41.889177,-87.638506,member


In [5]:
df_clean = combined_df.copy()
df_clean['started_at'] = pd.to_datetime(df_clean['started_at'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
df_clean['ended_at'] = pd.to_datetime(df_clean['ended_at'],format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

In [6]:
df_clean['ride_length_min'] =  (df_clean['ended_at'] - df_clean['started_at']).dt.total_seconds() / 60

df_clean['days_of _week_num'] = df_clean['started_at'].dt.day_of_week
df_clean['day_of_week'] = df_clean['started_at'].dt.day_name()

df_clean['month_num'] = df_clean['started_at'].dt.month
df_clean['month'] = df_clean['started_at'].dt.month_name()

df_clean['hour'] = df_clean['started_at'].dt.hour
df_clean['is_weekend'] = df_clean['days_of _week_num'].isin([5,6])

In [7]:
initial_rows = df_clean.shape[0]

neg_mask = df_clean['ride_length_min'] <= 0
cleaning_log['rows_removed_negative_duration'] = neg_mask.sum()
df_clean = df_clean[~neg_mask]

long_mask = df_clean['ride_length_min'] > 1440
cleaning_log['rows_removed_over_24h'] = long_mask.sum()
df_clean = df_clean[~long_mask]


cleaning_log['final_row_count'] = df_clean.shape[0]
cleaning_log['pct_removed'] = ((initial_rows - cleaning_log['final_row_count']) / initial_rows) * 100

print(f"Data Cleaning Complete. {cleaning_log['pct_removed']:.2f}% of data removed.")

Data Cleaning Complete. 0.09% of data removed.


In [8]:
clean_data_path = '../data/clean/trips_clean.parquet'
df_clean.to_parquet(clean_data_path, index=False, engine='pyarrow')
cleaning_log['output_files_saved'].append(clean_data_path)
print(f'Master cleaned data saved to {clean_data_path}.')

Master cleaned data saved to ../data/clean/trips_clean.parquet.


In [9]:

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
backup_path = f'./data/clean/backups/trips_clean_{timestamp}.parquet'
os.makedirs(os.path.dirname(backup_path), exist_ok=True) # Creates the /backups folder if it doesn't exist
df_clean.to_parquet(backup_path, index=False)
cleaning_log['output_files_saved'].append(backup_path)
print(f"Timestamped backup saved: {backup_path}")

Timestamped backup saved: ./data/clean/backups/trips_clean_20250902_194705.parquet


In [10]:
cleaning_log

{'initial_row_count': 5860568,
 'rows_removed_negative_duration': np.int64(43),
 'rows_removed_over_24h': np.int64(5142),
 'final_row_count': 5855383,
 'pct_removed': 0.0884726531626286,
 'output_files_saved': ['../data/clean/01_combined_raw.parquet',
  '../data/clean/trips_clean.parquet',
  './data/clean/backups/trips_clean_20250902_194705.parquet']}

In [11]:
log_content = f"""
# Data Cleaning Log
- Date & Time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
- Initial Number of Rides: {cleaning_log['initial_row_count']:,}
- Rides Removed (Negative Duration): {cleaning_log['rows_removed_negative_duration']:,}
- Rides Removed (>24 hours): {cleaning_log['rows_removed_over_24h']:,}
- Final Number of Rides: {cleaning_log['final_row_count']:,}
- Percentage of Data Removed: {cleaning_log['pct_removed']:.2f}%
- **Output Files Created:**
"""
for file in cleaning_log['output_files_saved']:
    log_content += f"  - `{file}`\n"

log_file_path = './data_cleaning_log.md'
with open(log_file_path, 'w') as f:
    f.write(log_content)
print(f"Cleaning log saved to: {log_file_path}")

print(log_content)

Cleaning log saved to: ./data_cleaning_log.md

# Data Cleaning Log
- Date & Time: 2025-09-02 19:47:12
- Initial Number of Rides: 5,860,568
- Rides Removed (Negative Duration): 43
- Rides Removed (>24 hours): 5,142
- Final Number of Rides: 5,855,383
- Percentage of Data Removed: 0.09%
- **Output Files Created:**
  - `../data/clean/01_combined_raw.parquet`
  - `../data/clean/trips_clean.parquet`
  - `./data/clean/backups/trips_clean_20250902_194705.parquet`



In [12]:

db_path = '../data/clean/cyclistic.db'
conn = sqlite3.connect(db_path)
print(f"Database connected: {db_path}")

df_clean.to_sql('cyclistic_trips', conn, if_exists='replace', index=False)
print("Data successfully loaded into SQL table 'cyclistic_trips'.")


conn.close()
print("Database connection closed.")

Database connected: ../data/clean/cyclistic.db
Data successfully loaded into SQL table 'cyclistic_trips'.
Database connection closed.
